In [1]:
import psycopg2
import pandas as pd 
import sqlalchemy as salc
import matplotlib
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import warnings
import datetime
import json
warnings.filterwarnings('ignore')

with open("../../config.json") as config_file:
    config = json.load(config_file)

database_connection_string = 'postgresql+psycopg2://{}:{}@{}:{}/{}'.format(config['user'], config['password'], config['host'], config['port'], config['database'])

dbschema='augur_data'
engine = salc.create_engine(
    database_connection_string,
    connect_args={'options': '-csearch_path={}'.format(dbschema)})

In [2]:
repolist = pd.DataFrame()

repo_query = salc.sql.text(f"""
            select rg_name, repo_group_id, repo_name, d.repo_id, repo_git, forked_from, repo_archived, count(*) as releases
            from (
             SELECT a.rg_name as rg_name,
                a.repo_group_id as repo_group_id,
                b.repo_name as repo_name,
                b.repo_id as repo_id,
                b.repo_git as repo_git, 
                b.forked_from as forked_from,
                b.repo_archived as repo_archived
            FROM
                repo_groups a,
                repo b
            WHERE
                a.repo_group_id = b.repo_group_id 
            ORDER BY
                rg_name,
                repo_name) d, releases c 
            where d.repo_id = c.repo_id 
            group by rg_name, repo_git, repo_group_id, repo_name, d.repo_id, forked_from, repo_archived
                

    """)

repolist = pd.read_sql(repo_query, con=engine)

display(repolist)

repolist.dtypes

,rg_name,repo_group_id,repo_name,repo_id,repo_git,forked_from,repo_archived,releases
0,18f,25602,10x-mlaas,125245,https://github.com/18f/10x-mlaas,Parent not available,1,1
1,18f,25602,18f-eslint,125466,https://github.com/18f/18f-eslint,Parent not available,1,3
2,18f,25602,18f-eslint-action,125473,https://github.com/18f/18f-eslint-action,Parent not available,1,2
3,18f,25602,18f.gsa.gov,124400,https://github.com/18f/18f.gsa.gov,Parent not available,0,14
4,18f,25602,18f-scaffolding,124778,https://github.com/18f/18f-scaffolding,Parent not available,1,1
...,...,...,...,...,...,...,...,...
39666,zotero,25444,zotero-libreoffice-integration,27205,https://github.com/zotero/zotero-libreoffice-i...,Parent not available,0,60
39667,zotero,25444,zotero-maps,27224,https://github.com/zotero/zotero-maps,schuyler/zotero-maps,0,1
39668,zotero,25444,zotero-standalone-build,27218,https://github.com/zotero/zotero-standalone-build,Parent not available,0,109
39669,zotero,25444,zotero-word-for-mac-integration,27192,https://github.com/zotero/zotero-word-for-mac-...,Parent not available,0,94


rg_name          object
repo_group_id     int64
repo_name        object
repo_id           int64
repo_git         object
forked_from      object
repo_archived     int64
releases          int64
dtype: object

In [3]:
repolist.to_csv(path_or_buf='releases_with_git_url.csv')